In [2]:
from bs4 import BeautifulSoup
import warnings
import requests
from datetime import datetime
# import pandas as pd
warnings.filterwarnings("ignore")

In [8]:
main_url ="https://edwardzou.blogspot.com/?fbclid=IwAR3HsqRaK0D4sbuDJX7agJwfQc2uLBtc9V0PRJR0uT3u3pGEF0F9AL3JVVw"
url = main_url
response = requests.get(url)
html = BeautifulSoup(response.text)
boxes = html.find("div", id="content-wrapper").find_all("div",class_="box-body-deg")



In [10]:
for box in boxes:
    print(box.find("a")["title"])
    print(box.find("a")["href"])

【邁向Apple大師之路】如何在Mac使用Windows 系統？Bootcamp VS 虛擬機
https://edwardzou.blogspot.com/2019/06/macUseWindows.html
【邁向Apple大師之路】Mac 入門及基本設定
https://edwardzou.blogspot.com/2019/06/macStart.html
【邁向Apple大師之路】iPhone怎麼快速充電?
https://edwardzou.blogspot.com/2019/04/iPhoneCharging.html
【日本自由行】岐阜縣飛驒高山古街,美麗的江戶時代風格街道
https://edwardzou.blogspot.com/2019/04/takayamastreet.html
【日本自由行】岐阜縣飛驒高山宮川朝市,日本三大早市之一
https://edwardzou.blogspot.com/2019/04/asaichi.html
【Angular】如何在token失效並重新登入後，重發前次Request?
https://edwardzou.blogspot.com/2019/04/angularResendRequest.html
【邁向Apple大師之路】不佔iCloud空間的共享相簿,優點與缺點分析
https://edwardzou.blogspot.com/2019/04/sharePhoto.html
【開箱文】匿名2 金色手搖磨豆機
https://edwardzou.blogspot.com/2019/03/coffeeGrinder.html
【食記】台北信義區 PAUL 法式麵包餐廳, 甜點不錯鹹食普普
https://edwardzou.blogspot.com/2019/03/paul.html
【教學】在iPhone上使用Suica西瓜卡
https://edwardzou.blogspot.com/2019/02/suica.html
【Angular】ngx-translate 多語系實務應用
https://edwardzou.blogspot.com/2019/01/ngx-translate.html
【邁向Apple大師之路】在iPhone 上限制App使用時間, iOS 12 的新功能
htt

In [ ]:



def crawler_main(pages, newest_update):
    main_url = "https://nba.udn.com/nba/index?gr=www"
    domain_url = "https://nba.udn.com"
    # go to main page > 焦點新聞 more
    newest_update = datetime.strptime(newest_update, "%Y-%m-%d %H:%M")
    url = main_url
    response = requests.get(url)
    html = BeautifulSoup(response.text)
    news_page = domain_url + html.find("div", id="mainbar").find("div", id="news").find("h1", class_="box-title").find("a")["href"]
    # https://nba.udn.com/nba/cate/6754
    # result at 0617
    # news_page = "https://nba.udn.com/nba/cate/6754"
    # go page1_newest
    news_articles = []
    for c_page in range(1, pages + 1):
        url = news_page + "/-1/newest/" + str(c_page)
        response = requests.get(url)
        html = BeautifulSoup(response.text)
        body_box = html.find("div", id="news_list_body").find("dl").find_all("dt")
        b = 0
        for box in body_box:
            url = domain_url + box.find("a")["href"]
            update_time = get_article_update_time(url)
            check_update_time = datetime.strptime(update_time, "%Y-%m-%d %H:%M")
            title = box.find("h3").text
            if check_update_time <= newest_update:
                b = 1
                break
            content = get_article_content(url)
            news_articles.append([url, title, update_time, content])
        if b:
            break
    return news_articles


def get_article_update_time(article_url):
    #     article_url = "https://nba.udn.com/nba/story/6780/3877002"
    response = requests.get(article_url)
    html = BeautifulSoup(response.text)
    aritle_update_time = html.find("div", id="shareBar").find("div", class_="shareBar__info") \
        .find("div", class_="shareBar__info--author").find("span").text
    return aritle_update_time


def get_article_content(article_url):
    #     aritle_url = "https://nba.udn.com/nba/story/6780/3877002"
    result = ""
    response = requests.get(article_url)
    html = BeautifulSoup(response.text)
    article_content = html.find("div", id="story_body_content").find_all("p")
    for c in article_content:
        result = result + c.text
    delete_content = " Getty Imagesfacebooktwitterpinterest"
    result = result.replace(delete_content, "")
    return result


# def main():
#     news = crawler_main(2, "2019-06-17 09:54")
#     df = pd.DataFrame(news, columns=["url", "title", "update_time","content"])
#     print(df)
#
#
# main()


In [5]:
from crawler_NBA import crawler_main
import warnings
from datetime import datetime
import sqlite3
warnings.filterwarnings("ignore")
# 先檢查有最新資料
conn = sqlite3.connect("../db.sqlite3")
c = conn.cursor()
sel = c.execute("select update_time from news order by update_time -1 limit 1")
conn.commit()
lastest_update = ""
for data in sel:
        lastest_update = data[0]
        print(lastest_update)
if lastest_update == "":
    lastest_update = "2019-06-15 00:00"
conn.close()

# 依照最新資料去爬
# default 爬最新的五頁 ~ 最新資料為止

news = crawler_main(2, lastest_update)
insert_data = []
print("total:", len(news), "data get")
for data in news:
    url, title, update_time, content = data
#     update_time = datetime.strptime(update_time, "%Y-%m-%d %H:%M")
    created = datetime.now().strftime("%Y-%m-%d %H:%M")
    insert_data.append((url, title, update_time, content,created,))

conn = sqlite3.connect("../db.sqlite3")
c = conn.cursor()
c.executemany('INSERT INTO news VALUES (null,?,?,?,?,?)', insert_data)
conn.commit()
conn.close()
print("data_updated")

total: 20 data get
data_updated


In [1]:
# # TRUNCATE TABLE
# from crawler_NBA import crawler_main
# import warnings
# from datetime import datetime
# import sqlite3
# warnings.filterwarnings("ignore")
# # 先檢查有最新資料
# conn = sqlite3.connect("../db_backup.sqlite3")
# c = conn.cursor()
# c.execute("DELETE FROM news")
# conn.commit()
# # for data in sel:
# #     lastest = data[0]
# conn.close()
# # print(lastest)